In [2]:
from gedcom.element.individual import IndividualElement
from gedcom.parser import Parser
import gedcom.tags
import numpy as np
import pandas as pd
import math

gedcom_parser = Parser()

In [3]:
def shortest_path(graph, v1, v2):
    """
    like shortest_path1, but more efficient
    as it maintains the border incrementally
    """

    # keep track of what has been visited
    # with what distance, and from what vertex
    visited = {v1: (0, None)}
    # the edges at the border between
    # the visited and unvisited parts
    border_edges = set()
    # the vertex that was just selected
    selected_vertex = v1

    while True:
        # add to the border the edges that
        # go out of the last selected vertex
        # to unvisited
        # print(f"{selected_vertex=}")
        adj = graph.get(selected_vertex, {})
        for (dest, weight) in adj.items():
            if dest not in visited:
                border_edges.add((selected_vertex, dest))
        # remove from the border any edge that would
        # end at the newly_elected vertex
        border_edges = {
            (s, d) for (s, d) in border_edges
            if d != selected_vertex
        }
        # print(f"{border_edges=}")

        # out of luck, no path can be found
        if not border_edges:
            print("no edges")
            return None

        # find the best tuple (edge, distance)
        shortest_length = math.inf
        shortest_edge = None
        for (s, d) in border_edges:
            w = graph[s][d]
            past_distance, _ = visited[s]
            dist = past_distance + w
            if dist <= shortest_length:
                shortest_length = dist
                shortest_edge = (s, d)

        # mark newly selected vertex
        best_src, best_dst = shortest_edge
        visited[best_dst] = (shortest_length, best_src)
        selected_vertex = best_dst

        # are we done ?
        if best_dst == v2:
            path = [v2]
            previous = best_src
            while previous:
                # print(f"inserting {previous}")
                path.insert(0, previous)
                previous = visited[previous][1]
            return shortest_length, path

In [4]:
def get_IndivFamily_DataFrame(file_path='Queen_Eliz_II.ged'):
    """
    Creates DataFrame of children & spouse families keys of all individuals 
    indexed by their keys, from a gedcom file.

    Parameters
    ---
    file_path : str
        path of the gedcom file

    Returns 
    ---
    pd.DataFrame 
        dataframe of children & spouse families keys of individuals
    """
    gedcom_parser.parse_file(file_path)
    root_child_elements = gedcom_parser.get_root_child_elements()
    
    T = []
    
    #Go through indivduals and get their families
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            L = [element.get_pointer()]
            for child_element in element.get_child_elements() :
                if child_element.get_tag() == gedcom.tags.GEDCOM_TAG_FAMILY_SPOUSE :
                    L += [child_element.get_value()]
                elif child_element.get_tag() == gedcom.tags.GEDCOM_TAG_FAMILY_CHILD :
                    L += [child_element.get_value()]
            T += [L]

    #Add NaN where information is missing
    full_T = [line+['NaN']*(3-len(line)) for line in T]

    #Create the DataFrame
    df = pd.DataFrame(
    {
        'INDI' : [full_T[k][0] for k in range(len(full_T))],
        'FAMS' : [full_T[k][1] for k in range(len(full_T))],
        'FAMC' : [full_T[k][2] for k in range(len(full_T))],
    })

    return df

In [10]:
get_IndivFamily_DataFrame()

,INDI,FAMS,FAMC
0,@I101@,@F285@,@F286@
1,@I103@,@F286@,@F287@
2,@I155@,@F78@,NaN
3,@I168@,@F75@,@F76@
4,@I169@,@F75@,@F209@
...,...,...,...
4678,@I11259@,@F6018@,@F6019@
4679,@I11260@,@F6019@,NaN
4680,@I11261@,@F6019@,NaN
4681,@I11262@,@F6020@,NaN


In [53]:
df = get_IndivFamily_DataFrame()
df1 = df.set_index('INDI',inplace=False)
df1.at['@I11262@','FAMC']

'NaN'

In [16]:
def get_FamLinks_DataFrame(file_path='Queen_Eliz_II.ged'):

    df = get_IndivFamily_DataFrame(file_path)

    df1 = df.set_index('INDI',inplace=False)

    df2 = pd.DataFrame(
    {
        'FAM' : df['FAMS'].drop_duplicates(),
    })
    df2['1FAMS'] = np.NaN
    df2.set_index('FAM',inplace=True)
    N_max = 1

    for family in df2.index :
        N = 0

        for indi in df[df['FAMS'] == f'{family}']['INDI'] :
            N += 1
            if N > N_max :
                N_max = N
                df2[f'{N_max}FAMS'] = np.NaN
            df2.at[f'{family}',f'{N}FAMS'] = f'{indi}'
            #df2.at[f'{family}',f'{N}FAMS'] = df1.at[f'{indi}','FAMC']
        
        for indi in df[df['FAMC'] == f'{family}']['INDI'] :
            N += 1
            if N > N_max :
                N_max = N
                df2[f'{N_max}FAMC'] = np.NaN
            df2.at[f'{family}',f'{N}FAMC'] = f'{indi}'
            #df2.at[f'{family}',f'{N}FAMC'] = df1.at[f'{indi}','FAMS']
    
    return df2

In [58]:
def get_IndivLinks_DataFrame(file_path='Queen_Eliz_II.ged'):

    df = get_IndivFamily_DataFrame(file_path)

    df1 = df.set_index('INDI',inplace=False)

    df2 = pd.DataFrame(
    {
        'INDI' : df1.index,
    })
    df2['PARENT1'] = np.NaN
    df2.set_index('INDI',inplace=True)

    for indiv1 in df2.index :

        #Parents : FAMC of the character = FAMS of parents
        N_parents = 0

        for indiv2 in df[df['FAMS'] == df1.at[f'{indiv1}','FAMC']]['INDI'] :
            N_parents += 1
            df2.at[f'{indiv1}',f'PARENT{N_parents}'] = f'{indiv2}'


        #Childrens : FAMS of the character = FAMC of childrens
        N_childrens = 0

        for indiv3 in df[df['FAMC'] == df1.at[f'{indiv1}','FAMS']]['INDI'] :
            N_childrens += 1
            df2.at[f'{indiv1}',f'CHILDREN{N_childrens}'] = f'{indiv3}'

        #Siblings : FAMC of character = FAMC of siblings
        N_siblings = 0

        for indiv4 in df[df['FAMC'] == df1.at[f'{indiv1}','FAMC']]['INDI'] :
            if indiv4 != indiv1 and df1.at[f'{indiv1}','FAMC'] != 'NaN':
                N_siblings += 1
                df2.at[f'{indiv1}',f'SIBLING{N_siblings}'] = f'{indiv4}'

    df2 = df2.reindex(sorted(df2.columns), axis=1)
    return df2

In [59]:
#pd.set_option('display.max_rows', None)
get_IndivLinks_DataFrame()

,CHILDREN1,CHILDREN2,CHILDREN3,CHILDREN4,CHILDREN5,PARENT1,PARENT2,SIBLING1,SIBLING2,SIBLING3,SIBLING4,SIBLING5
INDI,,,,,,,,,,,,
@I101@,NaN,NaN,NaN,NaN,NaN,@I103@,@I590@,NaN,NaN,NaN,NaN,NaN
@I103@,@I101@,NaN,NaN,NaN,NaN,@I591@,@I592@,NaN,NaN,NaN,NaN,NaN
@I155@,@I622@,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
@I168@,@I386@,NaN,NaN,NaN,NaN,@I170@,@I171@,NaN,NaN,NaN,NaN,NaN
@I169@,@I386@,NaN,NaN,NaN,NaN,@I431@,@I432@,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
@I11259@,@I11257@,NaN,NaN,NaN,NaN,@I11260@,@I11261@,NaN,NaN,NaN,NaN,NaN
@I11260@,@I11259@,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
@I11261@,@I11259@,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def build_FamGraph(file_path='Queen_Eliz_II.ged'):

    g = {}
    df = get_FamLinks_DataFrame(file_path)

    for FAM1 in df.index :
        g[FAM1] = {}
        for FAM2 in df.loc[FAM1] :
            if f'{FAM2}' != 'nan' :
                g[FAM1][f'{FAM2}'] = 1
    return g

In [8]:
shortest_path(build_FamGraph(), '@F76@', '@F6017@')

(28,
 ['@F76@',
  '@F75@',
  '@F187@',
  '@F189@',
  '@F190@',
  '@F191@',
  '@F742@',
  '@F2010@',
  '@F807@',
  '@F3665@',
  '@F4612@',
  '@F4611@',
  '@F4610@',
  '@F4613@',
  '@F4614@',
  '@F4615@',
  '@F4616@',
  '@F4617@',
  '@F4618@',
  '@F4619@',
  '@F5986@',
  '@F5974@',
  '@F5973@',
  '@F5972@',
  '@F5971@',
  '@F5970@',
  '@F6012@',
  '@F6013@',
  '@F6017@'])